In [ ]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib

In [2]:
# Numpy のアップグレード
!pip install numpy --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.24.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.24.3 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.3 which is incompatible.


ランタイムを再起動してください

In [1]:
# 環境変数を設定
!echo 'export LD_LIBRARY_PATH=/usr/lib' >> ~/.bashrc
!source ~/.bashrc

In [2]:
import pandas_datareader.data as pdr

def get_stock_data(code):
  df = pdr.DataReader("{}.JP".format(code), "stooq").sort_index()
  return df

In [3]:
import plotly.graph_objs as go
import talib as ta
import datetime as dt
import pandas as pd
import numpy as np

name = '積水ハウス'
code = 1928
df = get_stock_data(code)
close = df["Close"]

# 5日、25日移動平均の算出
ma5, ma25 = ta.SMA(close, timeperiod=5), ta.SMA(close, timeperiod=25)
df["ma5"], df["ma25"] = ma5, ma25

ma5, ma25 = df["ma5"], df["ma25"]
cross  = ma5 > ma25

cross_shift = cross.shift(1)
temp_gc = (cross != cross_shift) & (cross == True)
temp_dc  = (cross != cross_shift) & (cross == False)

# ゴールデンクロス発生日であればMA5の値、それ以外はNaN
gc = [m if g == True else np.nan for g, m in zip(temp_gc, ma5)]

# デッドクロス発生日であればMA25の値、それ以外はNan
dc = [m if d == True else np.nan for d, m in zip(temp_dc, ma25)]

# データフレームのカラムとして保存
df["gc"], df["dc"] = gc, dc

# ボリンジャーバンドの算出
upper2, _, lower2 = ta.BBANDS(close, timeperiod=25, nbdevup=2, nbdevdn=2, matype=0)
df['upper2'], df['lower2'] = upper2, lower2

# MACD、シグナル、ヒストグラムを算出
macd, macdsignal, hist = ta.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
df["macd"] = macd
df["macd_signal"] = macdsignal
df["hist"] = hist

# RSI
rsi14 = ta.RSI(close, timeperiod=14)
rsi28 = ta.RSI(close, timeperiod=28)
df["rsi14"], df["rsi28"] = rsi14, rsi28

# 補助線
df["70"],  df["30"] = [70 for _ in close], [30 for _ in close]

# ストキャスティクス
slowK, slowD = ta.STOCH(df["High"], df["Low"], df["Close"],
                                fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
df["slowK"], df["slowD"] = slowK, slowD

# 補助線
df["80"],  df["20"] = [80 for _ in close], [20 for _ in close]

rdf = df[dt.datetime(2021,11,1):dt.datetime(2022,3,31)]
rdf.index = pd.to_datetime(rdf.index).strftime("%m-%d-%Y")

layout = {
            "height": 1000,
            "title"  : { "text": "{} {}".format(code, name), "x":0.5 }, 
            "xaxis" : { "rangeslider": { "visible": False } },
            "yaxis1": { "domain": [.46, 1.0], "title": "価格（円）", "side": "left", "tickformat": "," },
            "yaxis2": { "domain": [.40, .46] }, 
            "yaxis3": { "domain": [.30, .395], "title": "MACD", "side": "right"}, # MACD
            "yaxis4": { "domain": [.20, .295], "title": "RSI", "side": "right"}, # RSI
            "yaxis5": { "domain": [.10, .195], "title": "STC", "side": "right"}, # ストキャスティクス
            "yaxis6": { "domain": [.00, .095], "title": "Volume", "side": "right"}, # 出来高
            "plot_bgcolor":"light blue"
          }

data =  [
            # ローソク足
            go.Candlestick(yaxis="y1", x=rdf.index, open=rdf["Open"], high=rdf["High"], low=rdf["Low"], close=rdf["Close"],   
                            increasing_line_color="red", decreasing_line_color="gray"),
         
            # 5日移動平均線
            go.Scatter(yaxis="y1", x=rdf.index, y=rdf["ma5"], name="MA5", line={ "color": "royalblue", "width": 1.2 } ),
         
            # 25日移動平均線
            go.Scatter(yaxis="y1", x=rdf.index, y=rdf["ma25"], name="MA25", line={ "color": "lightseagreen", "width": 1.2 } ),
         
            # ゴールデンクロス
            go.Scatter(yaxis="y1", x=rdf.index, y=rdf["gc"], name="Golden Cross", mode="markers", marker={ "size": 15, "color": "purple" }, opacity = 0.6),
         
            # デッドクロス
            go.Scatter(yaxis="y1", x=rdf.index, y=rdf["dc"], name="Dead Cross", mode="markers", marker={ "size": 15, "color": "black", "symbol": "x" },  opacity = 0.8),
            
            # ボリンジャーバンド
            go.Scatter(yaxis="y1", x=rdf.index ,y=rdf["upper2"], name= "", line={ "color": "lavender", "width": 0 }),
            go.Scatter(yaxis="y1", x=rdf.index ,y=rdf["lower2"], name= "BB", line={ "color": "lavender", "width": 0 }, 
                    fill="tonexty", fillcolor="rgba(170,170,170,.2)"),
         
            # MACD
            go.Scatter(yaxis="y3" ,x=rdf.index ,y=rdf["macd"], name= "macd", line={ "color": "magenta", "width": 1 } ),
            go.Scatter(yaxis="y3" ,x=rdf.index ,y=rdf["macd_signal"], name= "signal", line={ "color": "green", "width": 1 } ),
            go.Bar(yaxis="y3" ,x=rdf.index, y=rdf["hist"], name= "histgram", marker={ "color": "slategray" } ) ,

            # RSI
            go.Scatter(yaxis="y4" ,x=rdf.index ,y=rdf["rsi14"], name= "RSI14",line={ "color": "magenta", "width": 1 } ),
            go.Scatter(yaxis="y4" ,x=rdf.index ,y=rdf["rsi28"], name= "RSI28",line={ "color": "green", "width": 1 } ),
            go.Scatter(yaxis="y4" ,x=rdf.index ,y=rdf["30"], name= "30",line={ "color": "black", "width": 0.5 } ),
            go.Scatter(yaxis="y4" ,x=rdf.index ,y=rdf["70"], name= "70",line={ "color": "black", "width": 0.5 } ),

            # ストキャスティクス
            go.Scatter(yaxis="y5" ,x=rdf.index ,y=rdf["slowK"], name= "slowK",line={ "color": "magenta", "width": 1 } ),
            go.Scatter(yaxis="y5" ,x=rdf.index ,y=rdf["slowD"], name= "slowD",line={ "color": "green", "width": 1 } ),
            go.Scatter(yaxis="y5" ,x=rdf.index ,y=rdf["20"], name= "20",line={ "color": "black", "width": 0.5 } ),
            go.Scatter(yaxis="y5" ,x=rdf.index ,y=rdf["80"], name= "80",line={ "color": "black", "width": 0.5 } ),
         
            # 出来高
            go.Bar(yaxis="y6", x=rdf.index, y=rdf["Volume"], name= "Volume", marker={ "color": "slategray" } ) 
        ]

fig = go.Figure(layout = go.Layout(layout), data = data)
fig.update_layout({
  "xaxis":{
      "tickvals": rdf.index[::2], 
      "ticktext": ["{}-{}".format(x.split("-")[0], x.split("-")[1]) for x in rdf.index[::2]] 
      }
})
fig.show()